# Packages 

In [7]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
from math import *
from sklearn.metrics import r2_score
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 
%matplotlib inline
from tqdm import tqdm
import csv as csv

# Preliminary information

This script allows you the process raw PCR data by ddCT method. You first need to label all your conditions through Platypus GUI to get a label dataframe, converted in csv file.

Then, you need to define your path, experiment name, file name, and the number of plate if you have several plates.

You also need to define the method you want to use to normalize your data in the ddCT process. The default method use sthe mean of all untreated conditions, whatever conditions. You can also choose by writting on the cell below if you want to use one of the two other methods ;
    - global_minimal_value: it takes the minimal value of dCT amoung all your untreated samples
    - condition_minimal_value: it chooses the condition with the minimal mean between your samples amoug untreated conditions 

In [97]:
output_path = '/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_immunotron/'
experiment_name = '20230306_MHV_HeLa_kinetics'
name_rawdata_file = '20230202_MHV_HeLa_sybr_plate'
plate_number='3'
normalizing_method='default' # other possibilities: global_minimal_value or condition_minimal_value

# Process

In [106]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('{}/{}/Data/{}{}.txt'.format(output_path,experiment_name,name_rawdata_file,plate_number),sep='\t')
convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number))

#Import data
pcr_data_raw = pd.read_csv('{}/{}/Data/{}{}.csv'.format(output_path,experiment_name,name_rawdata_file,plate_number)).drop(['Unnamed: 0'],axis=1).rename(columns={'Pos':'Tag','Cp':'CT'})               
pcr_data = pcr_data_raw.dropna(axis=0, inplace=False)  

#delta2CT method
#mean replicate 

levels=PCR_data.index.names --> when we get multi index labelling from platypus
levels.remove['Replicate']
pcr_mean_data=PCR_data.groupby(by=levels).mean()

pcr_mean_data=PCR_data.groupby(by=['Titer','Timepoint','GeneType']).mean()

#dCT
goi_list=pcr_mean_data.index.get_level_values(level='GeneType').unique().drop('HK')
pcr_mean_2_data=pcr_mean_data.unstack(level=-1)
for goi in goi_list:
    pcr_mean_2_data['dCT'] = pcr_mean_2_data['CT',goi] - pcr_mean_2_data['CT','HK']


#ddCT
if normalizing_method=='global_minimal_value':
    DeltaCT_min=min(pcr_mean_2_data.query('Titer==0')['dCT'])
    print(DeltaCT_min)
# else if normalizing_method=='condition_minimal_value':
    # DeltaCT_min=min(
else:
    DeltaCT_min=pcr_mean_2_data.query('Titer==0')['dCT'].mean() #Baseline definition, could introduce several option, to be continued
    
pcr_mean_2_data['ddCT']=pcr_mean_2_data['dCT']-DeltaCT_min

#Gene expression
pcr_mean_2_data['GeneExpression']=2**(-pcr_mean_2_data['ddCT'])



PCR_data_mean.sort_values(['Titer','Timepoint'],inplace=True)
# PCR_data_mean.to_csv('{}/{}/Data/Analyzed_PCR_Data_Plate_{}.csv'.format(output_path,experiment_name,plate_number))

pcr_mean_2_data

CT         dCT       ddCT GeneExpression
GeneType          GOI    HK                                
Titer Timepoint                                            
0.0   1           7.5   6.5   1.0  -3.833333   1.425438e+01
      2           8.5   8.0   0.5  -4.333333   2.015874e+01
      3          43.0   7.0  36.0  31.166667   4.148570e-10
      4           7.5   7.5   0.0  -4.833333   2.850876e+01
      5           7.0  36.5 -29.5 -34.333333   2.164528e+10
...               ...   ...   ...        ...            ...
10.0  8           3.5   4.0  -0.5  -5.333333   4.031747e+01
      9           4.0   6.0  -2.0  -6.833333   1.140350e+02
      10          6.0   4.0   2.0  -2.833333   7.127190e+00
      11         49.0   2.0  47.0  42.166667   2.025669e-13
      12          4.5  28.0 -23.5 -28.333333   3.382075e+08

[96 rows x 5 columns]

In [17]:
#Convert txt to csv
convertion_to_csv = pd.read_csv('/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_hand/20230425_CVB_HeLa_kinetics_12h_Exp5/Data/CVB EXP5-042523 ALGA 12H KINETIC_TECHNIQUE_2.txt',sep='\t')
# convertion_to_csv.drop(['Include','Name','Concentration','Standard','Status'], axis=1,inplace=True)
convertion_to_csv.to_csv('/Users/lanauaa/Desktop/Viral_dynamics_project/Data_replication_hand/20230425_CVB_HeLa_kinetics_12h_Exp5/Data/CVB EXP5-042523 ALGA 12H KINETIC_TECHNIQUE_2.csv')

# GUI

In [19]:
import tkinter as tk
import pandas as pd
from tkinter import filedialog

class PCR_Analyzer_GUI:
    def __init__(self, master):
        self.master = master
        master.title("PCR Data Analysis")
        
        # create label for file path
        self.label_file_path = tk.Label(master, text="File Path:")
        self.label_file_path.grid(row=0, column=0, padx=5, pady=5)
        
        # create entry for file path
        self.entry_file_path = tk.Entry(master)
        self.entry_file_path.grid(row=0, column=1, padx=5, pady=5)
        
        # create button to browse for file
        self.button_browse_file = tk.Button(master, text="Browse", command=self.browse_file)
        self.button_browse_file.grid(row=0, column=2, padx=5, pady=5)
        
        # create dropdown menu for normalization method
        self.norm_method_options = ["Average of UNT samples","Minimum value amoung UNT samples", "Minimum value among the averages of  all conditions for UNT samples "]
        self.norm_method_var = tk.StringVar()
        self.norm_method_dropdown = tk.OptionMenu(master, self.norm_method_var, *self.norm_method_options)
        self.norm_method_dropdown.grid(row=1, column=1, padx=5, pady=5)
        self.norm_method_var.set(self.norm_method_options[0])
        self.norm_method_dropdown = tk.Label(master, text="Normalization method")
        self.norm_method_dropdown.grid(row=1, column=0, padx=5, pady=5)
        
        
        # create button to run script
        self.button_run_script = tk.Button(master, text="Run Script", command=self.run_script)
        self.button_run_script.grid(row=2, column=0, columnspan=3, padx=5, pady=5)
        
    def browse_file(self):
        # open file dialog box to select file
        file_path = filedialog.askopenfilename()
        
        # update entry with selected file path
        self.entry_file_path.delete(0, tk.END)
        self.entry_file_path.insert(0, file_path)
        
    def run_script(self):
        # get file path from entry
        file_path = self.entry_file_path.get()
        
        # get normalization method from dropdown
        normalizing_method = self.norm_method_var.get().replace(" ", "_").lower()

        
        # run script with provided file path
        PCR_data_raw = pd.read_csv(file_path).rename(columns={'Pos':'Tag','Cp':'CT'})
        PCR_data = PCR_data_raw.dropna(axis=0, inplace=False)
        
        pcr_mean_data=PCR_data.groupby(by=['Titer','Timepoint','GeneType']).mean()
        
        
        # dCT
        goi_list=pcr_mean_data.index.get_level_values(level='GeneType').unique().drop('HK')
        pcr_mean_2_data=pcr_mean_data.unstack(level=-1)
        for goi in goi_list:
            pcr_mean_2_data['dCT'] = pcr_mean_2_data['CT',goi] - pcr_mean_2_data['CT','HK']


        #d dCT
        # normalization option
        if normalizing_method=='global_minimal_value':
            DeltaCT_min=min(pcr_mean_2_data.query('Titer==0')['dCT'])
        # else if normalizing_method=='condition_minimal_value':
            # DeltaCT_min=min(
        else:
            DeltaCT_min=pcr_mean_2_data.query('Titer==0')['dCT'].mean() # baseline definition, could introduce several option, to be continued
        
        pcr_mean_2_data['ddCT']=pcr_mean_2_data['dCT']-DeltaCT_min

        # gene expression
        pcr_mean_2_data['GeneExpression']=2**(-pcr_mean_2_data['ddCT'])

        pcr_mean_2_data.sort_values(['Titer','Timepoint'],inplace=True)
        
        # open file dialog box to select output file path
        output_path = filedialog.asksaveasfilename(defaultextension=".csv")
        
        # output result to selected file path
        pcr_mean_2_data.to_csv(output_path)

# create GUI window
root = tk.Tk()
pcr_data_gui = PCR_Analyzer_GUI(root)
root.mainloop()